# Pull in the transliteration table

In [3]:
import requests

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
response = requests.get('https://scriptsource.org/cms/scripts/page.php?item_id=entry_detail&uid=vsytndbyev', headers=headers)

In [230]:
import pandas as pd

results = pd.read_html(response.text)
df = results[1]
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,NaN,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,NaN,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,NaN,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,NaN,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,NaN,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [24]:
df.Glyph = df.USV.apply(lambda val: chr(int(val, 16)))
df.head()

,Glyph,USV,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
0,ሀ,1200,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
1,ሁ,1201,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
2,ሂ,1202,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
3,ሃ,1203,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
4,ሄ,1204,hé,hē,he,hē,he,he,he,he,hē,hE


In [25]:
df.to_csv("raw/transliteration-table.csv", index=False)

# Simplify the transliteration table

In [109]:
df = pd.read_csv("raw/transliteration-table.csv")
df = df.drop(columns=['USV'])
df = df.set_index('Glyph')
df

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA
Glyph,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE
...,...,...,...,...,...,...,...,...,...,...
ᎎ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwē,pWE
ᎏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pW
ፘ,rya,NaN,NaN,NaN,NaN,rya,rya,NaN,~ri,NaN


In [110]:
import unidecode
import numpy as np

def simplify(value):
    try:
        return unidecode.unidecode(value).lower()
    except:
        return np.nan
    
simplified = df.applymap(simplify)
merged = df.join(simplified, rsuffix='_simp')
merged.head(10)

,ALA-LC,Alone-Stokes,B&D,Campbell,Chaîne,Cohen,Dawkins,Dept of State,ethiop,SERA,ALA-LC_simp,Alone-Stokes_simp,B&D_simp,Campbell_simp,Chaîne_simp,Cohen_simp,Dawkins_simp,Dept of State_simp,ethiop_simp,SERA_simp
Glyph,,,,,,,,,,,,,,,,,,,,
ሀ,ha,hā,hä,ha,hă,ha,hä,ha,ha,he,ha,ha,ha,ha,ha,ha,ha,ha,ha,he
ሁ,hu,hu,hu,hū,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu,hu
ሂ,hi,hī,hi,hī,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi,hi
ሃ,hā,hā,ha,hā,ha,ha,ha,ha,hā,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha,ha
ሄ,hé,hē,he,hē,he,he,he,he,hē,hE,he,he,he,he,he,he,he,he,he,he
ህ,he/h,h/hi,hə/hø,he,hĕ,h/hə,hï,hɪ,he,h,he/h,h/hi,h@/ho,he,he,h/h@,hi,hi,he,h
ሆ,ho,ho,ho,hō,ho,hʷo,ho,ho,ho,ho,ho,ho,ho,ho,ho,hwo,ho,ho,ho,ho
ለ,la,la,lä,la,lă,lɑ,lä,lə,la,le,la,la,la,la,la,la,la,l@,la,le
ሉ,lu,lu,lu,lū,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu,lu


In [134]:
def split_apart(key):
    return merged[key].str.split("/", expand=True).add_suffix('_' + key)

# Split apart he/h into two columns
colnames = list(merged.columns)
split = pd.concat([split_apart(col) for col in colnames], axis=1)
split

,0_ALA-LC,1_ALA-LC,0_Alone-Stokes,1_Alone-Stokes,0_B&D,1_B&D,2_B&D,3_B&D,0_Campbell,0_Chaîne,...,0_Campbell_simp,0_Chaîne_simp,0_Cohen_simp,1_Cohen_simp,2_Cohen_simp,0_Dawkins_simp,0_Dept of State_simp,0_ethiop_simp,0_SERA_simp,1_SERA_simp
Glyph,,,,,,,,,,,,,,,,,,,,,
ሀ,ha,None,hā,None,hä,None,None,None,ha,hă,...,ha,ha,ha,None,None,ha,ha,ha,he,None
ሁ,hu,None,hu,None,hu,None,None,None,hū,hu,...,hu,hu,hu,None,None,hu,hu,hu,hu,None
ሂ,hi,None,hī,None,hi,None,None,None,hī,hi,...,hi,hi,hi,None,None,hi,hi,hi,hi,None
ሃ,hā,None,hā,None,ha,None,None,None,hā,ha,...,ha,ha,ha,None,None,ha,ha,ha,ha,None
ሄ,hé,None,hē,None,he,None,None,None,hē,he,...,he,he,he,None,None,he,he,he,he,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ᎎ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pwe,None
ᎏ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pwe,pw,None
ፘ,rya,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,rya,None,None,rya,NaN,~ri,NaN,NaN


In [162]:
uniqued = df.fillna('').apply(lambda row: list(set([v for v in row.values if '/' not in str(v)])), axis=1)
uniqued

Glyph
ሀ    [hă, hä, he, hā, ha]
ሁ                [hu, hū]
ሂ                [hi, hī]
ሃ                [hā, ha]
ሄ        [hé, hE, hē, he]
             ...         
ᎎ            [, pwē, pWE]
ᎏ             [, pwe, pW]
ፘ            [, ~ri, rya]
ፙ            [, ~mA, mya]
ፚ            [, ~fi, fya]
Length: 368, dtype: object

In [166]:
uniqued.to_json("raw/lookup-table.json")

In [167]:
#merged.to_csv("raw/transliteration-table-large.csv")

# Use to transliterate

In [472]:
import json

with open("raw/lookup-table.json") as f:
    lookup = json.load(f)

punctuation = [
    ['።', '.'],
    ['፡', ' '],
    ['፣', ','],
    ['፤', ';'],
    ['፥', ':'],
    ['፧', '?']
]

for punc in punctuation:
    lookup[punc[0]] = [punc[1]]

lookup

{'ሀ': ['hă', 'hä', 'he', 'hā', 'ha'],
 'ሁ': ['hu', 'hū'],
 'ሂ': ['hi', 'hī'],
 'ሃ': ['hā', 'ha'],
 'ሄ': ['hé', 'hE', 'hē', 'he'],
 'ህ': ['h', 'hɪ', 'he', 'hï', 'hĕ'],
 'ሆ': ['hʷo', 'hō', 'ho'],
 'ለ': ['lă', 'lä', 'le', 'lɑ', 'la', 'lə'],
 'ሉ': ['lu', 'lū'],
 'ሊ': ['lī', 'li'],
 'ላ': ['la', 'lā'],
 'ሌ': ['lē', 'lʸe', 'le', 'lE', 'lé'],
 'ል': ['lï', 'le', 'l', 'lɪ', 'lĕ'],
 'ሎ': ['lo', 'lō', 'lʷo'],
 'ሏ': ['', 'lwa', 'lwā', 'lWa', 'lwä'],
 'ሐ': ['hä', 'ḥă', 'ḥa', 'ha', 'hā', 'He'],
 'ሑ': ['ḥū', 'ḥu', 'hu', 'Hu'],
 'ሒ': ['hi', 'ḥi', 'ḥī', 'Hi', 'hī'],
 'ሓ': ['ḥa', 'ha', 'hā', 'ḥā', 'Ha'],
 'ሔ': ['ḥē', 'ḥé', 'hē', 'he', 'ḥe', 'HE'],
 'ሕ': ['hɪ', 'H', 'hï', 'ḥĕ', 'ḥe'],
 'ሖ': ['ḥo', 'ho', 'hʷo', 'ḥō', 'Ho'],
 'ሗ': ['', 'ḥwā', 'HWa'],
 'መ': ['mă', 'mə', 'ma', 'mä', 'mɑ', 'me'],
 'ሙ': ['mū', 'mu'],
 'ሚ': ['mi', 'mī'],
 'ማ': ['ma', 'mā'],
 'ሜ': ['mē', 'mE', 'me', 'mʸe', 'mé'],
 'ም': ['mɪ', 'mĕ', 'mï', 'me', 'm'],
 'ሞ': ['mō', 'mo', 'mʷo'],
 'ᎀ': ['', 'mwa', 'mWe'],
 'ᎁ': ['', 'mWi', 'mwi'],
 '

### Generate a simplified version

In [473]:
lookup_simp = {}
for key in lookup.keys():
    values = list(set([unidecode.unidecode(v).lower() for v in lookup[key]]))
    lookup_simp[key] = [v for v in values if '@' not in v]
    
lookup_simp

{'ሀ': ['he', 'ha'],
 'ሁ': ['hu'],
 'ሂ': ['hi'],
 'ሃ': ['ha'],
 'ሄ': ['he'],
 'ህ': ['h', 'hi', 'he'],
 'ሆ': ['hwo', 'ho'],
 'ለ': ['la', 'le'],
 'ሉ': ['lu'],
 'ሊ': ['li'],
 'ላ': ['la'],
 'ሌ': ['lye', 'le'],
 'ል': ['le', 'li', 'l'],
 'ሎ': ['lo', 'lwo'],
 'ሏ': ['', 'lwa'],
 'ሐ': ['he', 'ha'],
 'ሑ': ['hu'],
 'ሒ': ['hi'],
 'ሓ': ['ha'],
 'ሔ': ['he'],
 'ሕ': ['hi', 'h', 'he'],
 'ሖ': ['hwo', 'ho'],
 'ሗ': ['', 'hwa'],
 'መ': ['ma', 'me'],
 'ሙ': ['mu'],
 'ሚ': ['mi'],
 'ማ': ['ma'],
 'ሜ': ['me', 'mye'],
 'ም': ['mi', 'me', 'm'],
 'ሞ': ['mwo', 'mo'],
 'ᎀ': ['', 'mwe', 'mwa'],
 'ᎁ': ['', 'mwi'],
 'ሟ': ['', 'mwa'],
 'ᎂ': ['', 'mwe'],
 'ᎃ': ['', 'mwe', 'mw'],
 'ሠ': ["'se", 'sa'],
 'ሡ': ['su', "'su"],
 'ሢ': ['si', "'si"],
 'ሣ': ["'sa", 'sa'],
 'ሤ': ['se', 'sye', "'se"],
 'ሥ': ['se', "'s", 'si'],
 'ሦ': ["'so", 'swo', 'so'],
 'ሧ': ['', "'swa", 'swa'],
 'ረ': ['ra', 're'],
 'ሩ': ['ru'],
 'ሪ': ['ri'],
 'ራ': ['ra'],
 'ሬ': ['re', 'rye'],
 'ር': ['ri', 're', 'r'],
 'ሮ': ['ro', 'rwo'],
 'ሯ': ['', 'rwa'],
 'ሰ': ['se'

In [474]:
import random
import more_itertools
    
replace_groups = [
    ['p', 'b'],
    ['k', 'c']
]

def reasonable_replacement(source, pct_chance=0.4):
    replaced = source
    for group in replace_groups:
        # 40% of the time maybe replace an item something else in the group
        replacer = lambda orig: random.choice(group) if random.random() < pct_chance else orig[0]
        replaced = re.sub("|".join(group), replacer, source)
    return replaced

def select_char(c, simple=False):
    try:
        if simple:
            return random.choice(lookup_simp[c])
        else:
            return random.choice(lookup[c])
    except:
        # Probably punctuation, we need to deal with that!
        return " "

def transliterate(word, simple=False, replacements=True):
    selected = ''.join([select_char(c, simple=simple) for c in word]).replace("  ", " ")
    if replacements:
        selected = reasonable_replacement(selected)
    return selected

Every time you run it, a different transcription

In [475]:
import re

text = """ማንም፡ሰው፡ቢሆን፡የጭካኔ፡ስቃይ፡እንዳይደርስበት፡
            ወይም፡ከሰብዓዊ፡አፈጻጸም፡ውጭ፡የሆነ፡የተዋረድ፡
            ተግባር፡ወይም፡ቅጣት፡አይፈጸምበትም።"""
text = re.sub(' +', ' ', text).replace("\n", "")

for _ in range(5):
    result = transliterate(text, simple=True)
    print(f"* {result}")

* manm sawi bihone jacicane sqaye 'enidayedarsebete weyim kasabi`awi 'afasasem wice jahwone jatawarede tegibar wayemi kitati ajefesamibateme.
* manme sew bihwone yechikane siqayi `enidaydersibet weyimi casebeawi `afatsatsam wice jahwona yatewarade tagbar weyime qtati 'ayifesambetime.
* manime sew bihwoni yecicane seqaje 'enidaydersibete waym cesabe'awi 'afetsasami wice yahona yatewaradi tagibare weyime qetati ayfesamebetem.
* manem sawe bihwon yekanye secay inidayederisbati wajemi cesabi'awi `afatsatsami wce jahwona yatewarad tegebar weym kitat 'ayefesemibatme.
* manemi sew bihoni jackanye sekayi `endajedaresibete wajem kasebe'awi `afesaseme wetkh'e yehwone yatawarad tagibari wayme kitati 'ayfesambatme.


`Simple=False` to allow accents and `@`

In [476]:
for _ in range(5):
    result = transliterate(text, simple=False)
    print(f"* {result}")

* mānĕm sɑwe bihone yəCkane sĕqajĕ inïdaydɑrïsĕbətɪ weymĕ cəsäbïāwī ʾăfəṣaṣämĕ wɪč̣e yəhōnä yätɑwarädɪ tegbarɪ wayme ḳïtātĕ ayfäṣämĕbătmï.
* manmɪ səwɪ bihōnï yaCɪkane sĕKajĕ ʾĕnedayderesĕbətĕ wäyïmɪ kesebɪʻāwī ʼafaṣāṣămɪ weCɪ jahōnă yetəwāradï təgbar wăjĕme ḳïṭātï ʾäyefaṣämbetïmĕ.
* manmï səwï bīhōnĕ jăCɪkāné sKaje ʼenɪdāydärɪsebäte weymï kəsɑbïāwi äfăSaṣämï wĕkḥï yähōnä yatɑwaradï təgĕbārï wăymï Kɪṭātï äyfătsambătɪm.
* manïmɪ səwĕ bīhōnĕ yatkh’ĕkāné sïKayï ʾĕnɪdajedarïsïbətï wäyem cɑsebïāwi ʾäfeṣāṣamï wĕ jăhōnɑ yətawārăde tägebare wajemĕ Kɪṭātï ʾäyefɑṣɑmɪbatïme.
* mānïmĕ săwĕ bihōn yekanē sɪḳayɪ ïnɪdāyɪdɑrɪsïbat wăyemĕ kăsäbɪʿawī ʼafăṣāṣamï wCɪ yəhōne yätəwārede tagĕbare wajĕmɪ KɪTatï ʾăyïfəṣämbätĕmĕ.


## Replacing similar items

In [477]:
import more_itertools
    
replace_groups = [
#     ['p', 'b'],
    ['k', 'c']
]

def reasonable_replacement_test(source, pct_chance=0.4):
    replaced = source
    for group in replace_groups:
        # 40% of the time maybe replace an item something else in the group
        replacer = lambda orig: random.choice(group) if random.random() < pct_chance else orig[0]
        replaced = re.sub("|".join(group), replacer, source)
    return replaced


for _ in range(5):
    print(reasonable_replacement_test("jacekane"))

jacekane
jakekane
jacekane
jacecane
jacekane


# Use it

Using data from `new-am.txt`, which is Bible + legal + news.

In [503]:
#text = open("corpuscrawler-master/corpus/am.txt").readlines()
text = open("raw/new-am.txt").readlines()
text = [line.strip() for line in text if len(line.strip()) > 0]
text[0:5]

['ጠ/ሚ መለስ ዜናዊ "ጦርነት ኳስ ጨዋታ አይደለም!" አሉ',
 'ሰሞኑን በሕወሓት/ኢሕአዴግ ግምገማ ውስጥ ዋነኛው የግምገማ በትር ያረፈው በጠ/ሚ መለስ ዜናዊ ላይ መሆኑ ተደጋግሞ እየተሰማ ነው።',
 'ከዚሁ ጋር ተያይዞ የጠ/ሚንስትሩ ጋርዶች በሌሎች መቀየራቸው፣  ከአቶ መለስ ዜናዊ ጋር የሚያገናኙ የቤተ መንግሥት የስልክ ግንኙነቶች መቋረጣቸው በሰፊው እየተነገረ ሲሆን፣ማንኛውንም የወቅቱን ጉዳይ አስመልክቶ መነጋገር የሚቻለው ከውጭ ጉዳይ ሚ/ር መ/ቤት ጋር መሆኑ ታውቋል።',
 'አቶ መለስ ዜናዊ በዚህ ሁኔታ ውስጥ መሆናቸው እየተነገረ ባለበት ወቅት ነው ከአሜሪካ ሬዲዮ ጋር ቃለ ምልልስ ያደረጉት።',
 'በቃለ ምልልሱም ወቅት በሕወሓት አመራር አካል የኤርትራን ጥያቄ ልዩነት ነበር ይባላል።']

Drop duplicates

In [504]:
uniques = list(set(text))
len(uniques)

48651

# Transliterate and save

In [497]:
uniques[:5]

['“የዳታ ፕሮሰሲንግ አገልግሎት” ማለት በኮምፒዩተር ሥርዓት አማካኝነት ዳታን የመቀበል፣ የማከማቸት፣ የመተንተን፣ የማሰራጨት፣ የማጓጓዝ ወይም የማስተላለፍ አገልግሎት ሲሆን የኔትዎርክ አገልግሎችንም',
 'የኤጀንሲው ዓላማ',
 'እንዲህ አለው፦ “ዮፍታሔ እንዲህ ይላል፦ ‘እስራኤል የሞዓባውያንን ምድርና የአሞናውያንን ምድር አልወሰደም፤',
 'ሆኖም ትርፏና የምትቀበለው ክፍያ ለይሖዋ የተቀደሰ ይሆናል። አይከማችም ወይም አይጠራቀምም፤ ምክንያቱም በይሖዋ ፊት የሚኖሩ ሰዎች እስኪጠግቡ ድረስ እንዲበሉና የሚያምር ልብስ እንዲለብሱ እሷ የምትቀበለው ክፍያ ለእነሱ ይሰጣል።',
 'በሀገራችን ኢትዮጵያ ተጨማሪ ጠላትና የሻዕቢያ ረዱት በመሆን አጋርነቱን እያረጋገጠ ለሚገኘው (ኦነግ) መጠናከርና ማደግ የሃይማኖት ሰዎች እጅ በድጋፍ ሰጪነት መኖሩ ተገለፀ።']

In [498]:
def separate_punc(text):
    replaced = text
    for punc in punctuation:
        for p in punc:
            if p != " ": 
                replaced = replaced.replace(p, f" {p} ")
    return re.sub(" +", " ", replaced).strip()

def clean_input(text):
    # We know what a space is and it makes things too complicated to use ፡
    cleaned = text.replace("፡", " ")
    return separate_punc(text)

clean_input(uniques[0])

'“የዳታ ፕሮሰሲንግ አገልግሎት” ማለት በኮምፒዩተር ሥርዓት አማካኝነት ዳታን የመቀበል ፣ የማከማቸት ፣ የመተንተን ፣ የማሰራጨት ፣ የማጓጓዝ ወይም የማስተላለፍ አገልግሎት ሲሆን የኔትዎርክ አገልግሎችንም'

In [499]:
NUM_TRANSLITERATIONS = 2

sources = []
transliterateds = []
for text in uniques:
    cleaned = clean_input(text)
    options = [transliterate(cleaned, simple=True) for _ in range(NUM_TRANSLITERATIONS)]
    for transliteration in options:
        sources.append(cleaned)
        transliterateds.append(transliteration)


print("originals look like", sources[0])
print("targets look like", transliterateds[0])

with open("raw/original.txt", 'w') as f:
    f.write('\n'.join(sources))

with open("raw/transliterated.txt", 'w') as f:
    f.write('\n'.join(transliterateds))

originals look like “የዳታ ፕሮሰሲንግ አገልግሎት” ማለት በኮምፒዩተር ሥርዓት አማካኝነት ዳታን የመቀበል ፣ የማከማቸት ፣ የመተንተን ፣ የማሰራጨት ፣ የማጓጓዝ ወይም የማስተላለፍ አገልግሎት ሲሆን የኔትዎርክ አገልግሎችንም
targets look like  jadata perwosasinegi `ageligelwote maleti becomepiyutari siri`at `amakannate datan yemecabal , jamacemaket , jamatanitan , yemasarat , yamagwagwaz wayim yemastelalafi 'agelglwoti sihwone yeneteworik agaligelonme


In [501]:
print("How many?", len(sources))

How many? 97302


In [502]:
!zip raw/datasets.zip raw/transliterated.txt raw/original.txt


updating: raw/transliterated.txt (deflated 58%)
updating: raw/original.txt (deflated 82%)


In [471]:
transliterate(clean_input("ከዓለም ዙሪያ የተውጣጡ ዜናዎችንና የተለያዩ መረጃዎችን በየዕለቱ እናቀርባለን። ምንጊዜም ከቢቢሲ አማርኛ ከማንም ያልወገኑ፣ ሚዛናዊ፣ ተዓማኒ፣"), simple=True)

"kaalami zuriya yetawetatu zyenawotchenina yatalajayu maradjawotchen bayailatu `enaqarebalani . mnegizeme kebibisi `amarna kemanim jaliwagenu , mizana'wi , teamani ,"

# Split into individual words

In [505]:
uniques[0:5]

['“የዳታ ፕሮሰሲንግ አገልግሎት” ማለት በኮምፒዩተር ሥርዓት አማካኝነት ዳታን የመቀበል፣ የማከማቸት፣ የመተንተን፣ የማሰራጨት፣ የማጓጓዝ ወይም የማስተላለፍ አገልግሎት ሲሆን የኔትዎርክ አገልግሎችንም',
 'የኤጀንሲው ዓላማ',
 'እንዲህ አለው፦ “ዮፍታሔ እንዲህ ይላል፦ ‘እስራኤል የሞዓባውያንን ምድርና የአሞናውያንን ምድር አልወሰደም፤',
 'ሆኖም ትርፏና የምትቀበለው ክፍያ ለይሖዋ የተቀደሰ ይሆናል። አይከማችም ወይም አይጠራቀምም፤ ምክንያቱም በይሖዋ ፊት የሚኖሩ ሰዎች እስኪጠግቡ ድረስ እንዲበሉና የሚያምር ልብስ እንዲለብሱ እሷ የምትቀበለው ክፍያ ለእነሱ ይሰጣል።',
 'በሀገራችን ኢትዮጵያ ተጨማሪ ጠላትና የሻዕቢያ ረዱት በመሆን አጋርነቱን እያረጋገጠ ለሚገኘው (ኦነግ) መጠናከርና ማደግ የሃይማኖት ሰዎች እጅ በድጋፍ ሰጪነት መኖሩ ተገለፀ።']

In [512]:
unique_words = list(set(' '.join(uniques).strip().split(' ')))
unique_words.remove('')
len(unique_words)

146557

In [513]:
unique_words[0:5]

['ስሜታቸውንና', 'እመድባለሁ፤', 'እሟገትልሻለሁ፤ደግሞም', 'የተባለን', 'የሚተነትኑ']

In [516]:
NUM_TRANSLITERATIONS = 4

sources = []
transliterateds = []
for text in unique_words:
    cleaned = clean_input(text)
    options = [transliterate(cleaned, simple=True) for _ in range(NUM_TRANSLITERATIONS)]
    for transliteration in options:
        sources.append(cleaned)
        transliterateds.append(transliteration)


print("originals look like", sources[0])
print("targets look like", transliterateds[0])

with open("raw/original-words.txt", 'w') as f:
    f.write('\n'.join(sources))

with open("raw/transliterated-words.txt", 'w') as f:
    f.write('\n'.join(transliterateds))

originals look like ስሜታቸውንና
targets look like smetacewinna'


In [517]:
!zip raw/datasets-words.zip raw/transliterated-words.txt raw/original-words.txt


updating: raw/transliterated-words.txt (deflated 62%)
updating: raw/original-words.txt (deflated 87%)
